In [ ]:
#facial emotion recognition in image
#print all emotion accuracy
#wan cheng de dai ma
#tou ming te xiao
#using PIL Draw
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from PIL import ImageFont, ImageDraw, Image
from keras.preprocessing.image import img_to_array
import numpy as np
import os
import cv2

emotion = {0 : 'Angry',
           1 : 'Disgust',
           2 : 'Fear',
           3 : 'Happy',
           4 : 'Normal',
           5 : 'Sad',
           6 : 'Surprise'}

path = 'C:\\Users\\CHENTIEJUN\\Desktop\\tt'
imgPath = os.path.join(path,'19.png')
outPath = os.path.join(path,'19_integrationDataset.png')

modelPath = 'E:\\network\\inceptionV3\\models\\inceptionV3_integrationDataset.h5'

detector = MTCNN()
model = load_model(modelPath)

image = cv2.cvtColor(cv2.imread(imgPath),cv2.COLOR_BGR2RGB)
image_copy = image.copy()
img = detector.detect_faces(image)

box = img[0]['box']
cropped = image[box[1]:box[1]+box[3],box[0]:box[0]+box[2]]

x = cv2.resize(cropped,(200,150),interpolation = cv2.INTER_AREA)
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
x /= 255
    
custom = model.predict(x)
maxIndex = np.argmax(custom)

mask_width = 130
box_height = 180
#font = cv2.LINE_AA
font = ImageFont.truetype('arial.ttf',15)
font_1 = ImageFont.truetype('arial.ttf',20)
_,img_width = image.shape[:2]

cv2.rectangle(image,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(67,116,217),1)

points = np.array([[box[0]+box[2]+5,box[1]],
                   [box[0]+box[2]+5+mask_width,box[1]],
                   [box[0]+box[2]+5+mask_width,box[1]+box_height],
                   [box[0]+box[2]+5,box[1]+box_height]],np.int32)

sub_img = image_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]
roi = image[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]

if (box[0]+box[2]+5+mask_width) > img_width:
    points = np.array([[box[0]+box[2]+5,box[1]],
                       [box[0]+box[2]+5+img_width,box[1]],
                       [box[0]+box[2]+5+img_width,box[1]+box_height],
                       [box[0]+box[2]+5,box[1]+box_height]],np.int32)

    sub_img = image_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+img_width]
    roi = image[box[1]:box[1]+box_height,box[0]+box[2]+5:img_width]
    
background = cv2.fillConvexPoly(image,points,(76,76,76))

dst = cv2.addWeighted(roi,0.5,sub_img,0.5,0)

background[points[0][1]:points[2][1],points[0][0]:points[2][0]] = dst
background = Image.fromarray(background)
draw = ImageDraw.Draw(background)

#cv2.putText(background,'Emotion:',(box[0]+box[2]+7,box[1]+15),font,0.6,(255,255,255),2,font)
draw.text((box[0]+box[2]+7,box[1]), 'Emotion:', font=font_1, fill=(255,255,255))
distance = 20
for i in range(len(emotion)):
    if i == maxIndex:
        #cv2.putText(background,emotion[i]+f':{custom[0][i] : .2%}',(box[0]+box[2]+9,box[1]+distance+15),font,0.5,(255,228,0),1,font)
        draw.text((box[0]+box[2]+9,box[1]+distance+15), emotion[i]+f':{custom[0][i] : .2%}', font=font, fill=(255,228,0))
    else:
        #cv2.putText(background,emotion[i]+f':{custom[0][i] : .2%}',(box[0]+box[2]+9,box[1]+distance+15),font,0.5,(255,255,255),1,font)    
        draw.text((box[0]+box[2]+9,box[1]+distance+15), emotion[i]+f':{custom[0][i] : .2%}', font=font, fill=(255,255,255))
    distance += 21

background = cv2.cvtColor(np.array(background),cv2.COLOR_RGB2BGR)
cv2.imshow('result',background)
cv2.imwrite(outPath,background)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#facial emotion recognition in Video
#tou ming xiao guo
#using PIL
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from PIL import ImageFont, ImageDraw, Image
from keras.preprocessing.image import img_to_array
import numpy as np
import os
import cv2

emotion = {0 : 'Angry',
           1 : 'Disgust',
           2 : 'Fear',
           3 : 'Happy',
           4 : 'Normal',
           5 : 'Sad',
           6 : 'Surprise'}

path = 'D:\Camera_data'
videoName = 'depthCam_Driver_10_10_23_59_31'

inputVideoPath = os.path.join(path,videoName)+'.avi'
outputVideoPath = path+'\\result'+videoName+'.avi'

modelPath = 'E:\\network\\inceptionV3\\models\\inceptionV3_integrationDataset.h5'

detector = MTCNN()
model = load_model(modelPath)

capture = cv2.VideoCapture(inputVideoPath)

fps = int(round(capture.get(cv2.CAP_PROP_FPS)))


width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(outputVideoPath,fourcc,fps,(width,height))

averageAccuracy = np.array([[0,0,0,0,0,0,0]])
flag = True
while flag:
    totalAccuracy = np.zeros((1,7))
    imageNum = 0
    for i in range(fps):
        ret,frame = capture.read()
        if not ret:
            flag = False
            break

        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        frame_copy = frame.copy()
        
        face = detector.detect_faces(frame)
        
        if not face:
            continue

        boxList = np.zeros(len(face))
        for i in range(len(face)):
            box = face[i]['box']
            boxList[i] = box[2]*box[3]

        maxBoxIndex = np.argmax(boxList)
        maxBox = face[maxBoxIndex]['box']

        cropped = frame[maxBox[1]:maxBox[1]+maxBox[3],maxBox[0]:maxBox[0]+maxBox[2]]
        
        x = cv2.resize(cropped,(200,150),interpolation = cv2.INTER_AREA)
        x = img_to_array(x)
        x = np.expand_dims(x, axis=0)
        x /= 255

        custom = model.predict(x)
        
        mask_width = 130
        _,img_width = frame.shape[:2]
        box_height = 180
        #font = cv2.LINE_AA
        font = ImageFont.truetype('arial.ttf',15)
        font_1 = ImageFont.truetype('arial.ttf',20)

        cv2.rectangle(frame,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(67,116,217),1)

        points = np.array([[box[0]+box[2]+5,box[1]],
                               [box[0]+box[2]+5+mask_width,box[1]],
                               [box[0]+box[2]+5+mask_width,box[1]+box_height],
                               [box[0]+box[2]+5,box[1]+box_height]],np.int32)

        sub_img = frame_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]
        roi = frame[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+mask_width]
        if (box[0]+box[2]+5+mask_width) > img_width:
            points = np.array([[box[0]+box[2]+5,box[1]],
                               [box[0]+box[2]+5+img_width,box[1]],
                               [box[0]+box[2]+5+img_width,box[1]+box_height],
                               [box[0]+box[2]+5,box[1]+box_height]],np.int32)

            sub_img = frame_copy[box[1]:box[1]+box_height,box[0]+box[2]+5:box[0]+box[2]+5+img_width]
            roi = frame[box[1]:box[1]+box_height,box[0]+box[2]+5:img_width]

        background = cv2.fillConvexPoly(frame,points,(76,76,76))

        dst = cv2.addWeighted(roi,0.5,sub_img,0.5,0)

        background[points[0][1]:points[2][1],points[0][0]:points[2][0]] = dst
        
        background = Image.fromarray(background)
        draw = ImageDraw.Draw(background)

        totalAccuracy += custom
        imageNum += 1
        
        if (imageNum != 0 and imageNum % 20 == 0):
            averageAccuracy = totalAccuracy / imageNum
            print(averageAccuracy)
            
        #cv2.rectangle(background,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(103,153,255),1)

        draw.text((box[0]+box[2]+7,box[1]), 'Emotion:', font=font_1, fill=(255,255,255))
        
        maxIndex = np.argmax(averageAccuracy)
        distance = 20
        
        if np.sum(averageAccuracy)==0:
            #cv2.putText(background,'Recognizing...',(box[0]+box[2]+9,box[1]+(box_height//2)),font,0.6,(255,228,0),1,cv2.LINE_AA)
            draw.text((box[0]+box[2]+9,box[1]+(box_height//2)), 'Recognizing...', font=font_1, fill=(255,228,0))    
        else:
            for i in range(len(emotion)):
                if i == maxIndex:
                    #cv2.putText(background,emotion[i]+f':{averageAccuracy[0][i] : .2%}',(box[0]+box[2]+7,box[1]+distance+15),font,0.5,(255,228,0),1,cv2.LINE_AA)
                    draw.text((box[0]+box[2]+7,box[1]+distance+15), emotion[i]+f':{averageAccuracy[0][i] : .2%}', font=font, fill=(255,228,0))
                else:
                    #cv2.putText(background,emotion[i]+f':{averageAccuracy[0][i] : .2%}',(box[0]+box[2]+7,box[1]+distance+15),font,0.5,(255,255,255),1,cv2.LINE_AA)    
                    draw.text((box[0]+box[2]+7,box[1]+distance+15), emotion[i]+f':{averageAccuracy[0][i] : .2%}', font=font, fill=(255,255,255))
                distance += 21

        background = cv2.cvtColor(np.array(background),cv2.COLOR_RGB2BGR)
        cv2.imshow('result',background)
        out.write(background)
    
    key = cv2.waitKey(3)
    if key == 27:
        break
        print('ese break...')
        
capture.release()
out.release()
cv2.destroyAllWindows()